#### 0

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 

In [2]:
np.random.seed(42)

X = np.random.rand(1000, 2) # can use uniform or rand, doesn't really matter.
X[0:5], X.shape

(array([[0.37454012, 0.95071431],
        [0.73199394, 0.59865848],
        [0.15601864, 0.15599452],
        [0.05808361, 0.86617615],
        [0.60111501, 0.70807258]]),
 (1000, 2))

In [3]:
epsilon = np.random.normal(0.1, size=1000)
epsilon.shape

(1000,)

In [4]:
X1 = np.array([X[i][0] for i in range(len(X))])
X2 = np.array([X[i][1] for i in range(len(X))])

y = 3 * X1 + 5 * X2 + 3 + epsilon
y.shape

(1000,)

In [5]:
X = np.c_[np.ones(len(X)), X]
X[0:3], X.shape

#### 1 a)

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape # X has two columns, so shape is different

((670, 2), (330, 2), (670,), (330,))

ValueError: non-broadcastable output operand with shape (2,1) doesn't match the broadcast shape (2,670)